In [52]:
import selenium
import pandas as pd
import numpy as np
import unittest
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import chrome
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score , confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import re

In [53]:

def convert_text_to_number(text):
  """Converts a text string of a large number with commas to a number.

  Args:
    text: The text string to convert.

  Returns:
    The number represented by the text string.
  """

  pattern = r"^(-?\d+)(\,\d+)?"
  match = re.match(pattern, text)
  if match:
    return int(match.group(1).replace(",", ""))
  else:
    return None

In [54]:
# Set up the Selenium web driver
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=options , service=Service(ChromeDriverManager().install()))


In [55]:
cities = ["נהריה", "אילת", "חיפה", "נתניה", "תל אביב","אשדוד","אשקלון","הרצליה","ירושלים","באר שבע"]
date_arr = []
street_arr = []
rooms_arr = []
area_size_arr = []
floor_arr = []
price_arr = []
price_per_sm_arr = []
year_arr = []
city_arr = []

for city in cities:
    driver.get('https://www.ad.co.il/nadlanprice?keyword=' + city)
    
    # Adjust the number of iterations in the range function to fetch more rows
    for _ in range(40):  # Fetching 40 rows per city (total 10,000 rows)
        element = driver.find_elements(By.XPATH, '//*[@id="cards"]/div/table/tbody/tr')
        for row in element:
            date = row.find_elements(By.TAG_NAME, "td")[0]
            ci = row.find_elements(By.TAG_NAME, "td")[1]
            street = row.find_elements(By.TAG_NAME, "td")[2]
            rooms = row.find_elements(By.TAG_NAME, "td")[3]
            area_size = row.find_elements(By.TAG_NAME, "td")[4]
            floor = row.find_elements(By.TAG_NAME, "td")[5]
            price = row.find_elements(By.TAG_NAME, "td")[6]
            price_per_sm = row.find_elements(By.TAG_NAME, "td")[7]
            year = row.find_elements(By.TAG_NAME, "td")[8]

            print(date.text, ci.text, street.text, rooms.text, area_size.text, floor.text, price.text, price_per_sm.text, year.text)
            date_arr.append(date.text)
            street_arr.append(street.text)
            rooms_arr.append(convert_text_to_number(rooms.text))
            area_size_arr.append(convert_text_to_number(area_size.text))
            floor_arr.append(convert_text_to_number(floor.text))
            price_arr.append(convert_text_to_number(price.text))
            if price_per_sm:
                price_per_sm_arr.append(price_per_sm.text)
            else:
                price_per_sm_arr.append(None)
            year_arr.append(convert_text_to_number(year.text))
            city_arr.append(city)
    
driver.quit()


24/1/2021 נהריה תל חי 28 4 107  1,380,000 12,897 2020
24/1/2021 נהריה בלפור 10 4 107 1 1,380,000 12,897 2020
22/1/2021 נהריה בלפור 26 5 140 4 1,500,000 10,714 2021
22/1/2021 נהריה אלי כהן 2 3 65  600,000 9,230 1970
22/1/2021 נהריה בלפור 26 5 140  1,500,000 10,714 2021
22/1/2021 נהריה וולפסון 74 3 65 4 600,000 9,230 1970
18/1/2021 נהריה שמאי 19 4 119  1,430,000 12,016 2020
18/1/2021 נהריה נהריה 10 4 119 5 1,430,000 12,016 2020
17/1/2021 נהריה דוד רמז 27 5 137  1,630,000 11,897 2015
17/1/2021 נהריה רמז 29 5 137 2 1,630,000 11,897 2015
17/1/2021 נהריה אפרים שריר 20 3 75 3 625,000 8,333 1960
17/1/2021 נהריה אפרים שריר 24 3 75  625,000 8,333 1960
14/1/2021 נהריה תל חי 28 4 109  1,289,400 11,829 2019
14/1/2021 נהריה תל חי 2 4 100  1,130,000 11,300 2021
14/1/2021 נהריה תל חי 28 5 131  961,305 7,338 2020
14/1/2021 נהריה בלפור 1 5 131 2 961,305 7,338 2020
14/1/2021 נהריה בלפור 10 4 100 3 1,130,000 11,300 2021
14/1/2021 נהריה נהריה 10 4 109 1 1,289,400 11,829 2019
14/1/2021 נהריה נהריה 10 5 122 

In [56]:
d = {'date': date_arr,
'rooms': rooms_arr,
'area_size': area_size_arr,
'floor': floor_arr,
'price': price_arr,
'year': year_arr,
'city': city_arr}

df = pd.DataFrame(data=d)

print(df)
df.to_csv('data.csv', index=True)


            date  rooms  area_size  floor  price    year     city
0      24/1/2021    4.0      107.0    NaN      1  2020.0    נהריה
1      24/1/2021    4.0      107.0    1.0      1  2020.0    נהריה
2      22/1/2021    5.0      140.0    4.0      1  2021.0    נהריה
3      22/1/2021    3.0       65.0    NaN    600  1970.0    נהריה
4      22/1/2021    5.0      140.0    NaN      1  2021.0    נהריה
...          ...    ...        ...    ...    ...     ...      ...
19635  14/1/2021    5.0      131.0    2.0      1  1995.0  באר שבע
19636  13/1/2021    5.0      196.0    1.0      2  2002.0  באר שבע
19637  13/1/2021    4.0       97.0    NaN    965  1993.0  באר שבע
19638  13/1/2021    3.0       70.0    1.0    590  1991.0  באר שבע
19639  13/1/2021    3.0       70.0    NaN    590  1991.0  באר שבע

[19640 rows x 7 columns]
